In [1]:
from dataset import MammographyDataset, create_categories, custom_collate
from torch.utils.data import DataLoader
import pandas as pd

from evaluation import *
from model import CustomFasterRCNN,FasterRCNNConfig
from train import Trainer

In [2]:
df = pd.read_csv("metadata/stratified_local.csv")
ZIP_PATH = "N:\\IDM Downloads\\Compressions\\vindr-mammo.zip"
INTER_NAME = "vindr-mammo-a-large-scale-benchmark-dataset-for-computer-aided-detection-and-diagnosis-in-full-field-digital-mammography-1.0.0/images"


In [2]:

all_categories,cat2idx = create_categories(df)

# Split into train and test based on fold
train_df = df[df['fold'] == 'training']
test_df = df[df['fold'] == 'test']



train_dataset = MammographyDataset(train_df, ZIP_PATH, INTER_NAME)
test_dataset = MammographyDataset(test_df, ZIP_PATH, INTER_NAME)

# Copy the category2idx and categories from the train dataset to the test dataset
# Because stratified with respect to birads and densities not findings
train_dataset.set_categories(all_categories, cat2idx)
test_dataset.set_categories(all_categories, cat2idx)

# Create dataloaders if needed
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True,collate_fn=custom_collate)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False,collate_fn=custom_collate)




In [3]:
config = FasterRCNNConfig()
model = CustomFasterRCNN(config=config)

C:\Users\user\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'backbone_name' and 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
C:\Users\user\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
trainer = Trainer(model,train_loader,test_loader,epochs=1,name="test")

In [5]:
trainer.validate()

KeyboardInterrupt: 